In [ ]:
# ─── 1. نصب کتابخانه‌های مورد نیاز ─────────────────────────────────────────────
!pip install statsmodels pandas --quiet

# ─── 2. آپلود فایل CSV به محیط Colab ──────────────────────────────────────────
from google.colab import files
uploaded = files.upload()

# ─── 3. خواندن و پاک‌سازی داده‌ها ────────────────────────────────────────────
import io
import pandas as pd

# فرض: اولین ستون تاریخ است
filename = list(uploaded.keys())[0]
df = pd.read_csv(
    io.BytesIO(uploaded[filename]),
    parse_dates=[0],
    index_col=0,
    thousands=','
)

# حذف ستون‌های کمکی TS.*
df.columns = df.columns.str.strip()
data = df.loc[:, ~df.columns.str.startswith('TS')]

# تبدیل همهٔ ستون‌ها به عددی و حذف سطرهای ناقص
data = data.apply(pd.to_numeric, errors='coerce').dropna()
print(f"داده نهایی: {data.shape[0]} سطر و {data.shape[1]} ستون")
print("متغیرها:", data.columns.tolist())

# ─── 4. آزمون Engle–Granger ───────────────────────────────────────────────────
from statsmodels.tsa.stattools import adfuller
import statsmodels.api as sm

# انتخاب Y و X (مثال: Y = Evaporation، X = بقیه)
Y = data['Evaporation']
X = data.drop(columns=['Evaporation'])

# افزودن ستون ثابت به ماتریس X
X_const = sm.add_constant(X)

# مرحله 1: رگرسیون OLS
ols_res = sm.OLS(Y, X_const).fit()
u = ols_res.resid

print("\n--- نتایج رگرسیون OLS ---")
print(ols_res.summary())

# مرحله 2: آزمون ADF روی باقیمانده‌ها
adf_res = adfuller(u, maxlag=1, regression='c')
print("\n--- آزمون ADF روی باقیمانده‌ها ---")
print(f"ADF Statistic: {adf_res[0]:.4f}")
print("Critical Values:")
for key, crit in adf_res[4].items():
    print(f"  {key}: {crit:.4f}")

if adf_res[0] < adf_res[4]['5%']:
    print("\nنتیجه: باقیمانده‌ها ایستا هستند → وجود همگرایی بین Evaporation و سایر متغیرها.")
else:
    print("\nنتیجه: باقیمانده‌ها ایستا نیستند → همگرایی تأیید نمی‌شود.")

# ─── 5. آزمون Phillips–Ouliaris (اختیاری) ─────────────────────────────────
from statsmodels.tsa.stattools import coint

coint_res = coint(Y, X)
print("\n--- آزمون Phillips–Ouliaris ---")
print(f"t-statistic: {coint_res[0]:.4f}")
print("Critical Values (90%, 95%, 99%):", coint_res[2])
if coint_res[0] < coint_res[2][1]:
    print("نتیجه: همگرایی تأیید می‌شود (t-stat < CV95).")
else:
    print("نتیجه: همگرایی تأیید نمی‌شود.")

Saving all.csv to all.csv


<ipython-input-1-e5238c71ac4d>:14: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(


داده نهایی: 165 سطر و 5 ستون
متغیرها: ['Evaporation', 'Precipitation', 'Ndvi', 'LST', 'Pm']

--- نتایج رگرسیون OLS ---
                            OLS Regression Results                            
Dep. Variable:            Evaporation   R-squared:                       0.037
Model:                            OLS   Adj. R-squared:                  0.013
Method:                 Least Squares   F-statistic:                     1.557
Date:                Sat, 07 Jun 2025   Prob (F-statistic):              0.188
Time:                        18:50:09   Log-Likelihood:                -936.16
No. Observations:                 165   AIC:                             1882.
Df Residuals:                     160   BIC:                             1898.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                    coef    std err          t      P>|t|      [0.025      0.975]
---------

#1. OLS Regression (Stage 1 of Engle–Granger)

You regressed Evaporation on the other four variables:

R-squared = 0.037 and Adj. R-squared = 0.013
Only about 1–4 % of the variation in Evaporation is explained by Precipitation, NDVI, LST, and Pm. This is very low, indicating a weak linear fit.

F-statistic = 1.557 (p = 0.188)
The overall regression is not statistically significant at conventional levels (e.g. α = 0.05). In other words, jointly the explanatory variables do not predict Evaporation significantly in this linear model.

Individual coefficients (e.g., Precipitation coef = –3.838, p = 0.176; NDVI coef = 4.452, p = 0.141; etc.)
None of the four slopes is individually significant (all p > 0.10).

Durbin–Watson = 0.71
This is far below 2.0, indicating positive autocorrelation in the residuals (a violation of OLS assumptions).

Condition Number ≈ 2.1 × 10⁹
Extremely large, suggesting severe multicollinearity or numerical ill-conditioning among regressors.

2. ADF Test on Residuals (Stage 2 of Engle–Granger)

You then tested whether the OLS residuals u^t\hat u_t are stationary:

ADF Statistic = –12.7783

Critical Values at 1%, 5%, 10% are –3.4711, –2.8794, –2.5763 respectively.

Because –12.78 is much lower (more negative) than all critical values, you reject the null of a unit root.
Conclusion: The residuals are stationary, which implies a cointegrating relationship exists between Evaporation and the other variables under the Engle–Granger framework.

3. Phillips–Ouliaris Test

This is another residual-based cointegration test:

t-statistic = –3.3803

Critical Values (90%, 95%, 99%) = –5.0927, –4.5013, –4.1972

Here, –3.38 is not more negative than the 95% or 90% critical values (it’s “closer” to zero).
Conclusion: You fail to reject the null of no cointegration according to Phillips–Ouliaris.

Overall Interpretation

Engle–Granger (ADF-on-residuals) says yes, the series are cointegrated (long-run equilibrium exists).

Phillips–Ouliaris says no, it does not find strong enough evidence for cointegration.

This mixed outcome sometimes happens when sample size is modest or when the residual-based tests have differing power or distributional assumptions. Given:

The very low R-squared and problematic diagnostics (autocorrelation, multicollinearity),

The strong ADF result on residuals, but the weaker Phillips–Ouliaris result,

you might:

Supplement with a different method (e.g. Johansen test, ARDL bounds test) for a more robust answer.

Examine the residual autocorrelation and heteroskedasticity—OLS-based cointegration tests assume well-behaved residuals.

Check each variable’s individual integration order (make sure none is I(2), which would invalidate these tests).

But strictly speaking, your Engle–Granger procedure suggests a long-run link, whereas the Phillips–Ouliaris test is inconclusive.